In [159]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic-sarcasm-detection/Mini Projet8.docx
/kaggle/input/arabic-sarcasm-detection/Sceintfic_Report_Model.pdf
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/LICENSE
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/README.md
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/training_data.csv
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/testing_data.csv


In [160]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [161]:
df = pd.read_csv('/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/training_data.csv')
print(df)
df_test = pd.read_csv('/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/testing_data.csv')
print(df_test)

                                                   tweet  sarcasm sentiment  \
0      "د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...    False       NEU   
1      "مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...    False       NEU   
2      “الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...     True       NEG   
3      "@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...     True       NEG   
4      "قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...    False       NEU   
...                                                  ...      ...       ...   
12543  هلأ صاير انت يا #فلعوط بدك تعطي محاضرات ع تويت...     True       NEG   
12544  "لا اله الا الله💜#أيفون_البروفيسور https://t.c...    False       NEU   
12545  "RT @turkyepost: #أردوغان : إذا كان المرتكب غي...    False       NEU   
12546  RT @Yousiif65: هاري بوتر؟👓🎩 https://t.co/959Oo...    False       POS   
12547        النت يلعن شكله طول الوقت تمام لين بدا الحفل     True       NEG   

      dialect  
0         msa  
1         msa  
2  

In [162]:
print(df.describe())

                                                    tweet sarcasm sentiment  \
count                                               12548   12548     12548   
unique                                              12487       2         3   
top     "RT @ahlalsunna2: #سوريا :الجيش السوري الحر يس...   False       NEU   
freq                                                    3   10380      5747   

       dialect  
count    12548  
unique       5  
top        msa  
freq      8562  


In [163]:
df.head(5)

,tweet,sarcasm,sentiment,dialect
0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa
1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa
2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa
3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf
4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa


In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12548 entries, 0 to 12547
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      12548 non-null  object
 1   sarcasm    12548 non-null  bool  
 2   sentiment  12548 non-null  object
 3   dialect    12548 non-null  object
dtypes: bool(1), object(3)
memory usage: 306.5+ KB


In [165]:
print(df.columns)

Index(['tweet', 'sarcasm', 'sentiment', 'dialect'], dtype='object')


In [166]:
labels, texts = df["sarcasm"], df["tweet"]
print(labels)
print(texts)

0        False
1        False
2         True
3         True
4        False
         ...  
12543     True
12544    False
12545    False
12546    False
12547     True
Name: sarcasm, Length: 12548, dtype: bool
0        "د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...
1        "مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...
2        “الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...
3        "@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...
4        "قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...
                               ...                        
12543    هلأ صاير انت يا #فلعوط بدك تعطي محاضرات ع تويت...
12544    "لا اله الا الله💜#أيفون_البروفيسور https://t.c...
12545    "RT @turkyepost: #أردوغان : إذا كان المرتكب غي...
12546    RT @Yousiif65: هاري بوتر؟👓🎩 https://t.co/959Oo...
12547          النت يلعن شكله طول الوقت تمام لين بدا الحفل
Name: tweet, Length: 12548, dtype: object


In [167]:
labels_test, texts_test = df_test["sarcasm"], df_test["tweet"]
print(labels_test)
print(texts_test)

0       False
1        True
2       False
3       False
4        True
        ...  
2995    False
2996    False
2997    False
2998    False
2999     True
Name: sarcasm, Length: 3000, dtype: bool
0       اخوي حانق يالغلا وشفيك معصب؟ عادي تراهم بشر يف...
1          اف مو متعوده عليهم سته https://t.co/8igFPx1i26
2                 اللهم اشفِ مرضانا ومرضى المسلمين . . ♥️
3                                    ابشركم طلقت السات 😘.
4       مؤشر خطير: ٩٠٪ من الشخصيات البرلمانية في الكوي...
                              ...                        
2995    تربكني الذكرى ليا مر طاريهوانسى البشر حولي وال...
2996                   وانا احسبهم الحين مايتركون حركاتهم
2997    #فههههههههههد_غشششششششاااااااام_الببببببصصصصصم...
2998    - لــو كــان الامـــر بيدي لأخفيت انهيار دموعي...
2999    َ نِسااء بابل كنّ يسْحرن الرّجل بالعيُون فقط ....
Name: tweet, Length: 3000, dtype: object


In [172]:

# Division des données en ensemble d'entraînement et ensemble de test
#X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.245, random_state=164)

X_train = df.drop('sarcasm', axis=1)  # Features for training
y_train = df['sarcasm']  # Labels for training

X_test = df_test.drop('sarcasm', axis=1)  # Features for testing
y_test = df_test['sarcasm']  # Labels for testing

print(X.shape)
print(X_train.shape)
print(X_test.shape)


(3000, 65270)
(12548, 3)
(3000, 3)


In [173]:
# Extraction des caractéristiques avec CountVectorizer
vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,2))
X = vectorizer.fit_transform(texts)
vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,2))
X = vectorizer.fit_transform(texts_test)
print(X)

  (0, 6351)	1
  (0, 28528)	1
  (0, 61850)	1
  (0, 58945)	1
  (0, 50750)	1
  (0, 36462)	1
  (0, 25410)	1
  (0, 22327)	1
  (0, 63880)	1
  (0, 13209)	1
  (0, 42135)	1
  (0, 58033)	1
  (0, 27296)	1
  (0, 5641)	1
  (0, 19845)	1
  (0, 48201)	1
  (0, 1645)	1
  (0, 6353)	1
  (0, 28529)	1
  (0, 61851)	1
  (0, 58946)	1
  (0, 50751)	1
  (0, 36469)	1
  (0, 25411)	1
  (0, 22329)	1
  :	:
  (2999, 60557)	1
  (2999, 40168)	1
  (2999, 60197)	1
  (2999, 32147)	1
  (2999, 21439)	1
  (2999, 33044)	1
  (2999, 42500)	1
  (2999, 32632)	1
  (2999, 20165)	1
  (2999, 43738)	1
  (2999, 62909)	1
  (2999, 28911)	1
  (2999, 11878)	2
  (2999, 27907)	1
  (2999, 20732)	1
  (2999, 60565)	1
  (2999, 40180)	1
  (2999, 32630)	1
  (2999, 18339)	1
  (2999, 60199)	1
  (2999, 62910)	1
  (2999, 32148)	1
  (2999, 27909)	1
  (2999, 21440)	1
  (2999, 33048)	1


In [180]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [181]:
# Debugging print statements
print("X_train_tfidf shape:", X_train_tfidf.shape)
print("Number of labels in y_train:", len(y_train))

X_train_tfidf shape: (3, 3)
Number of labels in y_train: 12548


In [183]:

model = MultinomialNB()
# model = ComplementNB()

# Création d'autre modèles
#model=tree.DecisionTreeClassifier()
#model=KNeighborsClassifier(n_neighbors=1)
#model = SVC(kernel="linear", C=1.0)
#model = MLPClassifier(hidden_layer_sizes=(5,), max_iter=20, activation = "logistic")


In [184]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Assuming X_train contains your DataFrame with a 'tweet' column
# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False)  # 'drop' parameter removes the first column to avoid multicollinearity

# Fit and transform the categorical data
one_hot_encoded = encoder.fit_transform(X_train[['tweet']])

# Get the feature names from the categories of the 'tweet' column
feature_names = encoder.get_feature_names_out(['tweet'])

# Convert the one-hot encoded array to a DataFrame
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=feature_names)

# Concatenate the one-hot encoded DataFrame with the original DataFrame
result = pd.concat([X_train, one_hot_encoded_df], axis=1)


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [185]:
model.fit(X_train, y_train)

ValueError: could not convert string to float: '"د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق رقم مهم في المعادلة السياسية المصرية ولا يمكن إغفالههل ترى أن هذا صحيح؟#أربعة_زائد_واحد"'

In [77]:

# Entraînement du modèle
model.fit(X_train, y_train)


ValueError: could not convert string to float: '"د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق رقم مهم في المعادلة السياسية المصرية ولا يمكن إغفالههل ترى أن هذا صحيح؟#أربعة_زائد_واحد"'

In [43]:

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)


ValueError: could not convert string to float: 'اخوي حانق يالغلا وشفيك معصب؟ عادي تراهم بشر يفطرون الصبح قلابة وتميس, تيك ات ايزي مان, RELAX  '

In [62]:

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

#Affichage des résultats
print(f"Précision du modèle: {accuracy:.2%}")
print("Matrice de confusion:\n", conf_matrix)

Précision du modèle: 83.32%
Matrice de confusion:
 [[2500   31]
 [ 482   62]]
